In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDERNAME = 'AI\ poster'
%cd drive/MyDrive/$FOLDERNAME

In [ ]:
# device
import torch, os
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)
print(os.getcwd())

In [ ]:
%pip install pydub
%pip install torchaudio

# for data preproccessing
from pathlib import PurePath
from pydub import AudioSegment
import shutil

# for loading data from drive
from torch.utils.data import DataLoader
import torchaudio.datasets as dset

# for creating waveform
import numpy as np
import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt

In [ ]:
# labeling
labels={}
with open('/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_cm_protocols/ASVspoof2019.PA.cm.train.trn.txt','r') as f:
  for line in f:
    if line.split()[-1] == 'spoof':
      labels[line.split()[1]]=line.split()[-1]
print(labels)

In [ ]:
wav_names = os.listdir('/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster/PA_T/ASVspoof2019_PA_train/wav')
wav_names=sorted(wav_names, reverse=True)

In [ ]:
train_path = '/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train'

In [ ]:
PA_T_wav_path = '/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster/PA_T/ASVspoof2019_PA_train/wav'

import wave

for wav_name in wav_names:
  if not os.path.exists(train_path+'/wav_train/spoof/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_val/spoof/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_val/bonafide/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_train/bonafide/'+wav_name[:len(wav_name)-4]+'.png'):
    os.chdir(PA_T_wav_path)

    if 'wav' in wav_name:
      wav_obj = wave.open(PA_T_wav_path+'/'+wav_name, 'rb')

      freq = wav_obj.getframerate() # Hz per sec
      n_freq = wav_obj.getnframes() # total Hz
      time = n_freq/freq      # lengh of the audio(s)
      # print(f'audio time: {time}')

      # channel check (1 channel)
      n_channels = wav_obj.getnchannels()
      # print(f'channel: {n_channels}')

      signal_wave = wav_obj.readframes(n_freq)
      signal_array = np.frombuffer(signal_wave, dtype=np.int16)

      # create waveform
      times = np.linspace(0, time, num=n_freq)
      plt.figure(figsize=(15, 5))
      plt.plot(times, signal_array)
      plt.title(wav_name)
      plt.xlabel('time(s)')
      plt.xlim(0, time)
      plt.ylabel('signal value')

      if labels[wav_name[:12]] == 'bonafide':
        os.chdir(train_path+'/wav_train/bonafide')
      else:
        os.chdir(train_path+'/wav_train/spoof')
      plt.savefig(wav_name[:len(wav_name)-4]+'.png')

    print(wav_name+' : '+labels[wav_name[:12]])